1. [Loading Data & Libraries](#1)
1. [Text Processing](#2)
1. [Build The Model](#3)
1. [Testing Model](#4)
1. [Model Training](#5)
1. [Extra: Customized Training](#6)

<a id = "1"></a><br>
# Loading Data & Libraries

In [1]:
import tensorflow as tf

import numpy as np
import os
import time
# To ignore warnings
import warnings
warnings.filterwarnings('ignore')

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [2]:
file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 [==============================] - 0s 0us/step


In [3]:
# Read, then decode for py2 compat.
text = open(file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 1115394 characters


In [4]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [5]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

65 unique characters


<a id = "2"></a><br>
# Text Processing

In [6]:
example = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [7]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

In [8]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

In [9]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [10]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [11]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [12]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [13]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [14]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

F
i
r
s
t
 
C
i
t
i


In [15]:
seq_length = 100
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


In [16]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [17]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [18]:
dataset = sequences.map(split_input_target)

In [19]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


<a id = "3"></a><br>
# Build The Model

In [20]:
# Batch size
BATCH_SIZE = 30

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(30, 100), dtype=tf.int64, name=None), TensorSpec(shape=(30, 100), dtype=tf.int64, name=None))>

In [21]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 100

In [22]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [23]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [24]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(30, 100, 66) # (batch_size, sequence_length, vocab_size)


In [25]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  16896     
                                                                 
 gru (GRU)                   multiple                  107400    
                                                                 
 dense (Dense)               multiple                  6666      
                                                                 
Total params: 130,962
Trainable params: 130,962
Non-trainable params: 0
_________________________________________________________________


<a id = "4"></a><br>
# Testing Model

In [26]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()
sampled_indices

array([22, 21,  1, 32, 62, 52, 64, 12, 55, 35, 49,  9, 40, 17, 19, 18, 18,
       57, 15, 33, 41, 17, 10, 14, 29, 39, 34, 37, 17, 19, 33, 14, 23, 36,
       17, 30, 57, 12, 11, 22, 56, 11, 15,  3, 11, 62, 11, 47, 15, 31, 20,
       61, 49, 10, 58, 40, 34, 51, 24, 21, 54, 29, 19, 15, 11, 60, 54, 19,
       36, 15,  5, 24, 13, 30,  7, 59, 39, 36, 32, 19, 16, 23,  3, 58, 49,
       65, 26, 54, 43, 62, 34, 37, 61, 47, 52, 25,  9, 14, 50, 12])

In [27]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'ak off the parley; for scarce I can refrain\nThe execution of my big-swoln heart\nUpon that Clifford, '

Next Char Predictions:
 b'IH\nSwmy;pVj.aDFEErBTbD3APZUXDFTAJWDQr;:Iq:B!:w:hBRGvj3saUlKHoPFB:uoFWB&K?Q,tZWSFCJ!sjzModwUXvhmL.Ak;'


In [28]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [29]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (30, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.191959, shape=(), dtype=float32)


In [30]:
tf.exp(example_batch_mean_loss).numpy()

66.15225

In [31]:
model.compile(optimizer='adam', loss=loss)

In [32]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

<a id = "5"></a><br>
# Model Training

In [33]:
EPOCHS = 20

In [34]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
368/368 [==============================] - 27s 65ms/step - loss: 2.5207
Epoch 2/20
368/368 [==============================] - 25s 65ms/step - loss: 1.9864
Epoch 3/20
368/368 [==============================] - 27s 69ms/step - loss: 1.8244
Epoch 4/20
368/368 [==============================] - 25s 65ms/step - loss: 1.7362
Epoch 5/20
368/368 [==============================] - 24s 62ms/step - loss: 1.6804
Epoch 6/20
368/368 [==============================] - 24s 63ms/step - loss: 1.6409
Epoch 7/20
368/368 [==============================] - 26s 67ms/step - loss: 1.6121
Epoch 8/20
368/368 [==============================] - 26s 67ms/step - loss: 1.5894
Epoch 9/20
368/368 [==============================] - 25s 65ms/step - loss: 1.5716
Epoch 10/20
368/368 [==============================] - 26s 66ms/step - loss: 1.5571
Epoch 11/20
368/368 [==============================] - 26s 67ms/step - loss: 1.5446
Epoch 12/20
368/368 [==============================] - 26s 68ms/step - loss: 1.5349
E

<a id = "6"></a><br>
# Extra: Customized Training

In [35]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [36]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [37]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ROMEO:
As your shame in the kings, far shall welcome up there curse her smorts
By heavens:
I will place opposed of depart most grace a house.

GLOUCESTER:
We have it as deny leek I aid dream that not.

KING LID:
Horse forth an it.
Now, dilly, God have ambell-begnian good, of Bick'd state?

CORIOLANUS:
This is ever may, nungly take
and
hath yea, day your suedd.

KING RICHARD III:
Loven, whis it is an redous. And her raught they will stay.

BENCY:
I know the hands all of and Wircure their comfort meanfaly, my lord, if you
are unfortune, I have cated, the scelfulled in a kingdow lovey
Throubled flower; the days his assess doth means
Lordy:
A gentleman is blessed kized
All a such
instrumention so benterspines not baral would love, she kings, nor her time, or it be amoin things like is thyself three any made, good againstly; peace; and you.
Is be your strevous may.
He not rocks, and not should nevery:
Brestielding:
But, like you go mer at unwair said withly being straight,
And reep.

Adwess

In [38]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b"ROMEO:\nLet unboled.\n\nLUCIO:\nHang effed.\n\nSOMEPESBY:\nWhile all the comence; as gives make the seem\nSignicangmimas waretwerets of your Aulan:\nDiskings\nFrom Engelo, it exprinets he relebs the tribuin.\n\nMENEN:\nWho soon no good death shall be is sweet could, it deturning for and glace of Nortute.\n\nROMEO:\nThe burinatioas, her prozaly friar most one gond it not scarding joy, they are word: but go, for my hadved with your prison'' myst all Welcome.\nAlazbly brother's so send tryst the hair, ear\nA right nay your common talk.\n\nBUNENELET:\nOn him.\n\nLUCIO:\nA please in men;\nBut two, and quutuoul is maidness;\nFor to cup them. Spalis.\nHow we well rishards.\n\nDUKE VINCENTIO:\nUpon my weap him cousin's poverty\nLasting in for the better to Speined before these disuse.\n\nKING RICHARD III:\nI preut.\n\nLEONTES:\nWell I are not be as sland\nCan a daughter to hephagea!' last\nIt blood furthing the a$perate.\nGeforate.\n\nLEONTES:\nAmvan of her that lives with cal wh

In [39]:
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs):
      inputs, labels = inputs
      with tf.GradientTape() as tape:
          predictions = self(inputs, training=True)
          loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}

In [40]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [41]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [42]:
model.fit(dataset, epochs=1)

368/368 [==============================] - 27s 66ms/step - loss: 2.5136


In [43]:
EPOCHS = 10

mean = tf.metrics.Mean()

for epoch in range(EPOCHS):
    start = time.time()

    mean.reset_states()
    for (batch_n, (inp, target)) in enumerate(dataset):
        logs = model.train_step([inp, target])
        mean.update_state(logs['loss'])

        if batch_n % 50 == 0:
            template = f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
            print(template)

    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print()
    print(f'Epoch {epoch+1} Loss: {mean.result().numpy():.4f}')
    print(f'Time taken for 1 epoch {time.time() - start:.2f} sec')
    print("_"*80)

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 2.1435
Epoch 1 Batch 50 Loss 2.0841
Epoch 1 Batch 100 Loss 2.0383
Epoch 1 Batch 150 Loss 2.0098
Epoch 1 Batch 200 Loss 1.9637
Epoch 1 Batch 250 Loss 1.9465
Epoch 1 Batch 300 Loss 1.9377
Epoch 1 Batch 350 Loss 1.8942

Epoch 1 Loss: 1.9892
Time taken for 1 epoch 26.49 sec
________________________________________________________________________________
Epoch 2 Batch 0 Loss 1.8623
Epoch 2 Batch 50 Loss 1.8902
Epoch 2 Batch 100 Loss 1.8546
Epoch 2 Batch 150 Loss 1.8609
Epoch 2 Batch 200 Loss 1.7816
Epoch 2 Batch 250 Loss 1.8320
Epoch 2 Batch 300 Loss 1.8073
Epoch 2 Batch 350 Loss 1.7784

Epoch 2 Loss: 1.8297
Time taken for 1 epoch 25.53 sec
________________________________________________________________________________
Epoch 3 Batch 0 Loss 1.7638
Epoch 3 Batch 50 Loss 1.7624
Epoch 3 Batch 100 Loss 1.7106
Epoch 3 Batch 150 Loss 1.7461
Epoch 3 Batch 200 Loss 1.7173
Epoch 3 Batch 250 Loss 1.7021
Epoch 3 Batch 300 Loss 1.7301
Epoch 3 Batch 350 Loss 1.6876

Epoch 3 Loss: 1.